In [113]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import pandas as pd
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
#TODO move this to a helpers folder and also why was a helper not provided anywhere?
def load_csv_data(data_path):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int64) #check if int 64 precision was actually needed
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    return yb, input_data, ids

In [115]:
df = pd.read_csv('train.csv')


In [116]:
print(df.columns[0])

Id


In [117]:
df.columns

Index(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [118]:
df['PRI_jet_num']

0         2
1         1
2         1
3         0
4         0
         ..
249995    0
249996    0
249997    1
249998    0
249999    0
Name: PRI_jet_num, Length: 250000, dtype: int64

In [119]:
import datetime

DATA_TRAIN_PATH = 'train.csv'
DATA_TEST_PATH = 'test.csv'
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


In [120]:
def clean_data(data):
    data_cleaned = data
    
    data_cleaned[data_cleaned == -999] = np.NaN
    
    #replace NaN's by mean of columns
    means = np.nanmean(data_cleaned, axis=0)
    print(means.shape)
    sq_std = np.std(data_cleaned, axis=0) **2
    inds = np.where(np.isnan(data_cleaned))
    data_cleaned[inds] = np.take(means, inds[1])
    
    #standardize the columns 
    data_cleaned = (data_cleaned - means)  / sq_std 
    
    
    
    return data 

In [121]:
tX_train_clean = clean_data(tX_train)
tX_test_clean = clean_data(tX_test)

(30,)
(30,)


In [122]:
print(y_train)

[ 1. -1. -1. ...  1. -1. -1.]


In [123]:
w, loss = ridge_regression(y_train, tX_train_clean, 0)

In [124]:
pred = tX_test_clean.dot(w)
pred = (pred - pred.min()) / (pred.max() - pred.min())

In [125]:
rounded_preds = np.round(pred, 0)

In [126]:
errors = np.sum(np.abs((y_test - rounded_preds)))

In [127]:
tot = y_test.shape[0]

In [128]:
(tot - errors) / tot

0.9823190282944823

In [129]:
y_test.dtype

dtype('float64')

In [130]:
def real_positives(pred, act):
    tot = 0
    good = 0
    for p,a in zip(pred,act):
        if(a == 1.0):
            tot += 1
            if(p == 1.0):
                good += 1
    return good / tot
                
            
def real_negatives(pred,act):
    tot = 1
    good = 0
    for p,a in zip(pred,act):
        if(a == 0.0):
            tot += 1
            if(p == 0.0):
                good += 1
    return good, tot, good / tot
        
    

In [131]:
real_positives(rounded_preds, y_test)

0.9823190282944823

In [132]:
real_negatives(rounded_preds, y_test)

(0, 1, 0.0)

In [133]:
zero_y_test = y_test[y_test == 0.0]

In [135]:
print(zero_y_test)

[]


In [136]:
y_test

array([1., 1., 1., ..., 1., 1., 1.])